It might be necessary to segment the images so as to extract some features to be used in classification.
Here is one of the ways to segment using Guassian Mixture Models.

Images are converted from RGB space to Lab color space
Smoothing is performed on 'a'
Then GMM is used to segment into various components.

I have used Initial code from a Notebook describing how to connect to coflax.

In [2]:
import os
import platform


abspath_dataset_dir_train_1 = '/data/kaggle/train/Type_1'
abspath_dataset_dir_train_2 = '/data/kaggle/train/Type_2'
abspath_dataset_dir_train_3 = '/data/kaggle/train/Type_3'
abspath_dataset_dir_test    = '/data/kaggle/test/'

# For kaggle kernels
if 'c001' not in platform.node():
    abspath_dataset_dir_train_1 = '../input/train/Type_1'
    abspath_dataset_dir_train_2 = '../input/train/Type_2'
    abspath_dataset_dir_train_3 = '../input/train/Type_3'
    abspath_dataset_dir_test    = '../input/test/'

    
def get_list_abspath_img(abspath_dataset_dir):
    list_abspath_img = []
    for str_name_file_or_dir in os.listdir(abspath_dataset_dir):
        if ('.jpg' in str_name_file_or_dir) == True:
            list_abspath_img.append(os.path.join(abspath_dataset_dir, str_name_file_or_dir))
    list_abspath_img.sort()
    return list_abspath_img


list_abspath_img_train_1 = get_list_abspath_img(abspath_dataset_dir_train_1)
list_abspath_img_train_2 = get_list_abspath_img(abspath_dataset_dir_train_2)
list_abspath_img_train_3 = get_list_abspath_img(abspath_dataset_dir_train_3)
list_abspath_img_test    = get_list_abspath_img(abspath_dataset_dir_test)

list_abspath_img_train        = list_abspath_img_train_1 + list_abspath_img_train_2 + list_abspath_img_train_3
list_answer_dataset_dir_train = [1] * len(list_abspath_img_train_1) + [2] * len(list_abspath_img_train_2) + [3] * len(list_abspath_img_train_3)

In [3]:
import cv2
import matplotlib.pyplot
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

def sub_func_load_img(abspath_img):
    img_rgb = cv2.cvtColor(cv2.imread(abspath_img), cv2.COLOR_BGR2RGB)
    return cv2.resize(img_rgb,(0,0), fx=0.05, fy=0.05)

def show_img(abspath_img):
    matplotlib.pyplot.imshow(sub_func_load_img(abspath_img))
    matplotlib.pyplot.show()

In [33]:
rgb = sub_func_load_img(list_abspath_img_train[252])
lab = cv2.cvtColor(rgb,cv2.COLOR_RGB2LAB)

imglab = cv2.medianBlur(lab,41)
np.random.seed(1)
imgdata = imglab[:,:,1]#Take 'a' component
flatImgL = imgdata.flatten()
g = GaussianMixture(n_components=2,covariance_type='diag')
g.fit(flatImgL[:,np.newaxis])
label_data = g.predict(flatImgL[:,np.newaxis])
label_data= label_data.reshape(imgdata.shape)

ndx = label_data[:,:] == 1
ndx = ndx.astype(float)

mask = np.zeros_like(rgb)
for i in np.arange(3):
    mask[:,:,i]= ndx
    
plt.imshow(rgb)
plt.show()
segImg = np.multiply(rgb,mask)
plt.imshow(np.multiply(rgb,mask))
plt.show()

In [34]:
chans = cv2.split(segImg)
colors = ("r", "g", "b")
plt.title("'Flattened' Color Histogram")
plt.xlabel("Bins")
plt.ylabel("# of Pixels")
features = []
for (chan, color) in zip(chans, colors):
    hist = cv2.calcHist([chan], [0], None, [256], [2, 256])
    features.extend(hist)
    # plot the histogram
    plt.plot(hist, color = color)
    plt.xlim([0, 256])


In [35]:
# import the necessary packages
from skimage import feature
import numpy as np
 
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
 
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, 149))
			
 
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
 
		# return the histogram of Local Binary Patterns
		return hist
# initialize the local binary patterns descriptor along with
# the data and label lists

desc = LocalBinaryPatterns(24,3)
plt.title("Histogram of LBP")
plt.xlabel("Bins")
plt.ylabel("# of Pixels")
features = []
for (chan, color) in zip(chans, colors):
    hist = desc.describe(chan)
    # plot the histogram
    plt.plot(hist, color = color)
    plt.xlim([0, 30])

